In [2]:
from dataclasses import dataclass

@dataclass
class MAPConfigs:
    model_id: str
    num_agents: int
    

configs = MAPConfigs(
    model_id="Qwen/Qwen2.5-1.5B-Instruct",
    num_agents=2,
)

: 

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer



# Pick a sensible dtype for your hardware
dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
device_map = "auto"

tok = AutoTokenizer.from_pretrained(configs.model_id, use_fast=True)
def init_model():
    return AutoModelForCausalLM.from_pretrained(
        configs.model_id,
        torch_dtype=dtype,
        device_map=device_map,
    )

models = [init_model() for _ in range(configs.num_agents)]

/home/ubuntu/multi-agent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import openai
from openai import OpenAI

def inference(model, system_prompt, problem, max_tokens=1024, temperature=0.7):
    client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"  
    )

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem}
        ],
        max_tokens=max_tokens,
        temperature=temperature
    )

    return resp.choices[0].message.content



In [25]:

#how should we structure this?
#allow the models to communicate then RL them on Math?
#let's start super early with that 
system_prompt = """
You are in a mulitagent env where you can communicate with eachother to help solve it together
to communicate you can type <send> Your message to the other agent </send>
Messages received from other agents will be show up in your context as <received>Message Sent </received>
You are encourage to message often to help collobratively solve the problem together
You should do some working out and send over what findings you have that you think would be helpful to the other agent
"""

problem = """
Solve the following MATH problem
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
"""

reprompt = """
This is the response from the env
"""

model = "meta-llama/llama-3.2-1b-instruct"



@dataclass
class Message:
    content:str
    model_generated: bool
    tokenized: list = None

messages = []


messages.append(Message(system_prompt, False))
messages.append(Message(problem, False))

output = inference(model, system_prompt, problem)

messages.append(Message(output, True))


In [26]:
formatted_follow_up = """
{message}

<recieved>
{output}
</recieved>
"""

output = inference(model, system_prompt, formatted_follow_up)
output
messages.append(Message(output, True))

In [38]:

back_prop_outputs = [message.content for message in messages if message.model_generated]
for message in messages:
    message.tokenized = tok(message.content).input_ids
    
output_ids = []
for message in messages:
    output_ids.extend(message.tokenized)

output_ids_tensor = torch.tensor(output_ids)

tensor([ 6.4688,  8.1250, 12.0000,  ..., -1.4688, -1.4688, -1.4688],
       dtype=torch.bfloat16, grad_fn=<ViewBackward0>)

In [ ]:
logits = models[0](output_ids_tensor.unsqueeze(0)).loss["logits"].view(-1)

backprop_logits = []

index = 0 

for message in messages:
    if message.model_generated:
        
       
        backprop_logits.append(logits[index: index + len(message.tokenized)])
        
    index += len(message.tokenized)

loss = torch.tensor(-1 * backprop_logits)
loss.backwards()



tensor([12.0000,  8.0625,  6.8125], dtype=torch.bfloat16,
       grad_fn=<SliceBackward0>)
tensor([12.0000,  8.0625,  6.8125], dtype=torch.bfloat16,
       grad_fn=<SliceBackward0>)


In [62]:
from dataclasses import dataclass
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import openai
from openai import OpenAI

@dataclass
class MAPConfigs:
    model_id: str
    num_agents: int
    
configs = MAPConfigs(
    model_id="Qwen/Qwen2.5-1.5B-Instruct",
    num_agents=2,
)

# Pick a sensible dtype for your hardware
dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
device_map = "auto"
tok = AutoTokenizer.from_pretrained(configs.model_id, use_fast=True)

def init_model():
    return AutoModelForCausalLM.from_pretrained(
        configs.model_id,
        torch_dtype=dtype,
        device_map=device_map,
    )

models = [init_model() for _ in range(configs.num_agents)]  # Fixed the syntax error

@dataclass
class Message:
    content: str
    model_generated: bool
    token_ids: torch.Tensor = None  # Store as tensor directly
    start_idx: int = None  # Track position in combined sequence
    end_idx: int = None

def inference(model, system_prompt, problem, max_tokens=1024, temperature=0.7):
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key="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"  
    )
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem}
        ],
        max_tokens=max_tokens,
        temperature=temperature
    )
    return resp.choices[0].message.content

def tokenize_and_combine_messages(messages, tokenizer):
    """
    Tokenize messages and create combined tensor while preserving gradient flow
    Returns combined tensor and updates message objects with indices
    """
    all_token_tensors = []
    current_idx = 0
    
    for message in messages:
        # Tokenize to tensor directly
        tokens = tokenizer(message.content, return_tensors="pt").input_ids.squeeze(0)
        message.token_ids = tokens
        message.start_idx = current_idx
        message.end_idx = current_idx + len(tokens)
        
        all_token_tensors.append(tokens)
        current_idx += len(tokens)
    
    # Concatenate all tensors (preserves gradients)
    combined_tokens = torch.cat(all_token_tensors, dim=0)
    return combined_tokens

def extract_model_generated_logits(logits, messages):
    """
    Extract logits for model-generated messages while preserving gradients
    """
    backprop_logits = []
    
    for message in messages:
        if message.model_generated:
            # Slice logits using tensor indexing (preserves gradients)
            message_logits = logits[message.start_idx:message.end_idx]
            backprop_logits.append(message_logits)
    
    return backprop_logits

# Main execution
system_prompt = """
You are in a multiagent env where you can communicate with each other to help solve it together
to communicate you can type <send> Your message to the other agent </send>
Messages received from other agents will be shown up in your context as <received>Message Sent </received>
You are encouraged to message often to help collaboratively solve the problem together
You should do some working out and send over what findings you have that you think would be helpful to the other agent
"""

problem = """
Solve the following MATH problem
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
"""

model = "meta-llama/llama-3.2-1b-instruct"

# Initialize messages
messages = []
messages.append(Message(system_prompt, False))
messages.append(Message(problem, False))

# Get first response
output1 = inference(model, system_prompt, problem)
messages.append(Message(output1, True))

# Format follow-up
formatted_follow_up = f"""
{problem}
<received>
{output1}
</received>
"""

# Get second response
output2 = inference(model, system_prompt, formatted_follow_up)
messages.append(Message(output2, True))

# Tokenize and combine while preserving gradients
combined_tokens = tokenize_and_combine_messages(messages, tok)

# Forward pass through model
model_output = models[0](combined_tokens.unsqueeze(0))
logits = model_output.logits.squeeze(0)  # Remove batch dimension

# Extract logits for model-generated content (preserves gradients)
backprop_logits = extract_model_generated_logits(logits, messages)

# Compute loss (example: negative log likelihood)
total_loss = torch.tensor(0.0, requires_grad=True)

for message_logits in backprop_logits:
    # Example loss computation - you'll want to replace this with your actual loss
    # This is just a placeholder - you need proper target tokens for real loss
    loss_contribution = -message_logits.sum()  # Placeholder loss
    total_loss = total_loss + loss_contribution

# Now you can backpropagate!
total_loss.backward()

print(f"Total loss: {total_loss.item()}")
print("Gradients preserved and computed successfully!")

# Alternative approach using cross-entropy loss if you have target tokens
def compute_proper_loss(logits, target_tokens):
    """
    Compute proper cross-entropy loss for language modeling
    """
    # Shift logits and targets for next-token prediction
    shift_logits = logits[:-1].contiguous()
    shift_labels = target_tokens[1:].contiguous()
    
    # Flatten for cross-entropy
    shift_logits = shift_logits.view(-1, shift_logits.size(-1))
    shift_labels = shift_labels.view(-1)
    
    # Compute loss
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(shift_logits, shift_labels)
    return loss

# If you have proper target sequences, use this instead:
# for i, message in enumerate(messages):
#     if message.model_generated:
#         message_logits = logits[message.start_idx:message.end_idx]
#         target_tokens = message.token_ids
#         loss = compute_proper_loss(message_logits, target_tokens)
#         loss.backward(retain_graph=True)  # Use retain_graph for multiple backwards

Total loss: 58916864.0
Gradients preserved and computed successfully!


In [70]:
import textarena as ta

env = ta.make(env_id="TicTacToe-v0")


In [72]:


#tool use 


def inference(model, system_prompt, problem, max_tokens=1024, temperature=0.7):
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key="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"  
    )
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem}
        ],
        max_tokens=max_tokens,
        temperature=temperature
    )
    return resp.choices[0].message.content

def call_inference_tool(prompts):
    MODEL = "meta-llama/llama-3.2-1b-instruct"
    system_prompt= ""
    
    for prompt in prompts:
        inference(MODEL, system_prompt=system_prompt, prompt=prompt)

    



In [76]:
prompt= """

You are an orchestrator for a mulitiagent RL system. Your goal is to develop a pipeline for finding the corect solution
You have up to 10 agents working for you. 

You can call each one by stating 
<prompt agent=1>Solve the following</prompt>

The models response will then be put in context like this
<response>Models Response</response> 
DON"T INCLUDE RESPONSE IN YOUR OUTPUT JSUT output all the prompts you want then i'll reprompt you with the real resposne

You can set the system prompt for each with
<system> Model System prompt e.g you are a erally skilled mathmatician</system>

Here's the problem you have to collobratively solve
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$


"""
output = inference("meta-llama/llama-3.2-3b-instruct", "", prompt)

In [81]:
def extract_prompts(text):
    index = 0
    pairs = []
    while text.find("<prompt", index):
        left = text.find("<prompt", index)
        right = text.find("prompt>", left)
        if right == -1: break
        index = right
        print(right)
        pairs.append(text[left:right])
    
    return pairs

prompts = extract_prompts(output)

outputs = [inference("meta-llama/llama-3.2-3b-instruct", "", prompt) for prompt in prompts]

180
337
493
666
849
1040
1236
1444
1612
1791


In [82]:
outputs

['To convert the point (0,3) in rectangular coordinates to polar coordinates, we can use the following formulas:\n\nr = √(x^2 + y^2)\n\nθ = arctan(y/x)\n\nPlugging in the values x = 0 and y = 3, we get:\n\nr = √(0^2 + 3^2) = √(0 + 9) = √9 = 3\n\nθ = arctan(3/0)\n\nSince arctan(3/0) is undefined, we need to consider the following:\n\n* If x = 0, then θ = π/2 (if y > 0) or -π/2 (if y < 0)\n* Since y = 3 > 0, we choose θ = π/2\n\nSo, the polar coordinates of the point (0,3) are (3, π/2).',
 'To convert from rectangular (Cartesian) coordinates to polar coordinates, we use the formulas:\n\nr = sqrt(x^2 + y^2)\n\nθ = arctan(y/x)\n\nwhere r is the radius (distance from the origin), θ is the angle in radians, x is the horizontal coordinate, and y is the vertical coordinate.\n\nIn this case, the point is (0,3), so:\n\nr = sqrt(0^2 + 3^2) = sqrt(0 + 9) = sqrt(9) = 3\n\nθ = arctan(3/0) \n\nSince the arctan function returns an angle in radians, we get:\n\nθ = π/2 (90 degrees)',
 'To convert the po

In [6]:
import torch
rewards = torch.arange(10).float()
rewards

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [9]:
(rewards - torch.mean(rewards))/torch.std(rewards)

tensor([-1.4863, -1.1560, -0.8257, -0.4954, -0.1651,  0.1651,  0.4954,  0.8257,
         1.1560,  1.4863])

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "gpt2"  # or "gpt2-medium", "gpt2-large", "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
ref_model = GPT2LMHeadModel.from_pretrained(model_name)



In [33]:
expected_output = torch.ones([1, 3, tokenizer.vocab_size]).float()
expected_output

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [17]:
input_ids = tokenizer.encode("Hello World!")
input_ids = torch.tensor(input_ids)

In [27]:
ouputs = model(input_ids.unsqueeze(0)).logits
probs = torch.nn.Softmax(dim=-1)(ouputs)

In [29]:
probs/ probs

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], grad_fn=<DivBackward0>)

In [10]:

rewards = torch.ones(4)

float_rewards = rewards.float()
reward_mean = torch.mean(float_rewards)
reward_std = torch.std(float_rewards)

advantage = (float_rewards -reward_mean)/reward_std
advantage= torch.nan_to_num(advantage, 0)
advantage

tensor([0., 0., 0., 0.])

In [11]:
len(advantage)

4

In [ ]:

model_probs = torch.tensor([])
ref_model_probs = torch.tensor([])
advantage = torch.tensor([])

float_rewards = rewards.float()
reward_mean = torch.mean(float_rewards)
reward_std = torch.std(float_rewards)

advantage = (float_rewards -reward_mean)/reward_std


formatted_string = torch.tensor([rollout.format_conversation_str for rollout in rollouts])
inputs_ids = torch.tensor([tokenizer.encode(string) for string in  formatted_string])



model_logits = torch.concat(model(inputs_ids))
ref_model_logits = torch.concat(ref_model(inputs_ids))

model_probs = torch.log(model_logits)
ref_model_probs = torch.log(ref_model_logits)

ratios = model_probs/ ref_model_probs

clipped_min = torch.clip(ratios, 1-eps, 1+eps)

loss = torch.min(ratios, clipped_min)




In [1]:
len(advantage)

NameError: name 'advantage' is not defined